# Import libraries (~10-30 minutes)

In [1]:
!pip install "torch==2.5.1" "triton==3.1.0" "recbole==1.2.0" "causal-conv1d==1.5.3.post1" "einops==0.8.1" "numpy==1.26.4" gdown

  Installing build dependencies ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

# Download dataset

In [2]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1WDDhDgDlIz-uEVknbDakwhirytwQ1upS?usp=sharing")

Retrieving folder contents


Retrieving folder 1z43FDT5Kg4lK7AKG4CPoly1axJJsKg9b dataset
Retrieving folder 1yGe8bCqgiXnncFgaEvabBhoHkpD_KibN amazon-beauty
Processing file 1_ZFZuLEBnmoxpYw0niYRS0Q9ozLcMC6Y amazon-beauty.inter
Retrieving folder 18U3LdjIDWOSb-V5JMAUiYBKMqT-IsMys ml-1m
Processing file 16QYqHl42FV4gFMVHHWmavqRjKFbSU2Wz ml-1m.inter
Retrieving folder 1rGWRpgEJQYNDLWNvWfmZUXhtfQJVMW-l yelp
Processing file 1r-F7HDKxycURAOaOpyF7OfppyPGkNyNe yelp.inter
Processing file 1zOp4lXf9e6uZUu7RBxa6iL3-vAdXnFC3 bert4rec.py
Processing file 13RQJuGZH7FCbQVoYDluznjQwPNcAHoew compare_plots.py
Processing file 1fUySZCs7y5uNcw6_IfsSkOUCdUOnteFf config.yaml
Processing file 1r3bcYZhiO78xX2P1yqU0rc4uNgzUUM5J parallel_scan.py
Processing file 1n2lJpZWX1eORMEbmfdkjbe4QYPXwcF-V plot_utils.py
Processing file 1wRJapKgN2LOtq2ZlcF2RSIaesY_3z7ZD README.md
Processing file 1OHZJ-ZLZrZYTSgkNa1qshZ0DVD6NyXln RecBLR.py
Processing file 1gHfSmNn5nVdUUFz6Byzv1I-HqUVNoeyo run.py
Processing file 1Lmerziu73569I3JlqYA8c8eiMt8z2GgS sasrec.py


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_ZFZuLEBnmoxpYw0niYRS0Q9ozLcMC6Y
To: /root/UPDATED_STRUCTURE/dataset/amazon-beauty/amazon-beauty.inter
100%|█████████████████████████████████████████████████████████████████████| 82.4M/82.4M [00:02<00:00, 38.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=16QYqHl42FV4gFMVHHWmavqRjKFbSU2Wz
To: /root/UPDATED_STRUCTURE/dataset/ml-1m/ml-1m.inter
100%|█████████████████████████████████████████████████████████████████████| 21.6M/21.6M [00:00<00:00, 36.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r-F7HDKxycURAOaOpyF7OfppyPGkNyNe
To: /root/UPDATED_STRUCTURE/dataset/yelp/yelp.inter
100%|█████████████████████████████████████████████████████████████████████| 61.5M/61.5M [00:01<00:00, 51.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1zOp4lXf9e6uZUu7RBxa6iL3-vAdXnFC3
From (redirected): https://d

['/root/UPDATED_STRUCTURE/dataset/amazon-beauty/amazon-beauty.inter',
 '/root/UPDATED_STRUCTURE/dataset/ml-1m/ml-1m.inter',
 '/root/UPDATED_STRUCTURE/dataset/yelp/yelp.inter',
 '/root/UPDATED_STRUCTURE/bert4rec.py',
 '/root/UPDATED_STRUCTURE/compare_plots.py',
 '/root/UPDATED_STRUCTURE/config.yaml',
 '/root/UPDATED_STRUCTURE/parallel_scan.py',
 '/root/UPDATED_STRUCTURE/plot_utils.py',
 '/root/UPDATED_STRUCTURE/README.md',
 '/root/UPDATED_STRUCTURE/RecBLR.py',
 '/root/UPDATED_STRUCTURE/run.py',
 '/root/UPDATED_STRUCTURE/sasrec.py']

# Guide
- model=R -> RecBLR
- model=B -> BERT4Rec
- model=S -> SASRec
- Note: BERT4Rec cost more VRAM to run, could get OOM error if we use 16GB VRAM of T4
- Time: SASRec > RecBLR > BERT4Rec

- Change dataset in the yaml file in the below cell. List: amazon-beauty, yelp, ml-1m

# Utility function

In [3]:
import os

def gen_config(
    bd_lru_only: bool = False,
    disable_conv1d: bool = False,
    disable_ffn: bool = False,
    num_layers: int = 2,
    num_epochs: int = 100,
    file_path: str = "/root/UPDATED_STRUCTURE/config.yaml",
    dataset: str = "amazon-beauty"
) -> None:
    """
    Generates the configuration content dynamically based on input parameters
    and writes it to the specified YAML file path.

    Args:
        bd_lru_only: Whether to use BD-LRU only mode. (bool)
        disable_conv1d: Whether to disable the 1D convolution block. (bool)
        disable_ffn: Whether to disable the FFN block. (bool)
        num_layers: Number of Recurrent layers. (int)
        file_path: The path to which the configuration file should be written. (str)
    """

    yaml_content = f"""
gpu_id: '0'

# RecBLR architecture flags
bd_lru_only: {bd_lru_only}
disable_conv1d: {disable_conv1d}
disable_ffn: {disable_ffn}

# RecBLR settings
hidden_size: 64         # (int) Number of features in the hidden state.
num_layers: {num_layers}  # (int) Number of Recurrent layers.
dropout_prob: 0.2       # (float) Dropout rate.
loss_type: 'CE'         # (str) Type of loss function. Range in ['BPR', 'CE'].
expand: 2               # (int) Block expansion factor
d_conv: 4               # (int) Local convolution width

# dataset settings
dataset: {dataset}
MAX_ITEM_LIST_LENGTH: 200    

USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
load_col:
    inter: [user_id, item_id, timestamp]

user_inter_num_interval: "[5,inf)"
item_inter_num_interval: "[5,inf)"

# training settings
epochs: {num_epochs}
train_batch_size: 2048
learner: adam
learning_rate: 0.001
eval_step: 1
stopping_step: 10
train_neg_sample_args: ~

# evalution settings
metrics: ['Hit', 'NDCG', 'MRR']
valid_metric: NDCG@10
eval_batch_size: 4096
weight_decay: 0.0
topk: [10, 20]
"""

    # --- Writing the file ---
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    try:
        with open(file_path, 'w') as f:
            f.write(yaml_content)
        print(f"Successfully wrote dynamically generated configuration to {file_path}")

    except Exception as e:
        print(f"Error writing file {file_path}: {e}")
        return

# RecBLR

In [4]:
MODEL='R'
EPOCHS=100
dataset = "yelp"
# dataset = "ml-1m"

## Default

In [5]:
gen_config(num_epochs=EPOCHS, dataset=dataset)        # default is 100, we should follow that
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model R] will not be used in RecBole
09 Nov 05:59    INFO  ['run.py', '--model', 'R']
09 Nov 05:59    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


# Unseen Item Handling Experiments

The following sections test RecBLR with preprocessing and/or postprocessing to handle unseen items:

1. **Preprocessing Only**: Maps unseen items in input sequences to their most similar known items based on content similarity. This ensures RecBLR processes clean, in-vocabulary sequences.

2. **Postprocessing Only**: Extends prediction scores from known items to unseen items using weighted similarity. This allows recommendations to cover the entire catalog.

3. **Both (Full Pipeline)**: Applies both preprocessing and postprocessing for comprehensive unseen item handling.

**Note**: These experiments require item features. For datasets without item metadata, synthetic features will be created based on interaction patterns.

In [ ]:
gen_config(num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run_with_unseen.py --model $MODEL --mode both

## Default + Preprocessing + Postprocessing (Full Unseen Handling)

In [ ]:
gen_config(num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run_with_unseen.py --model $MODEL --mode postprocessing

## Default + Postprocessing Only (Extend Scores to Unseen Items)

In [ ]:
gen_config(num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run_with_unseen.py --model $MODEL --mode preprocessing

## Default + Preprocessing Only (Handle Unseen Items in Input)

## Single Recurrent Layer

In [6]:
gen_config(num_layers=1, num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model R] will not be used in RecBole
09 Nov 06:10    INFO  ['run.py', '--model', 'R']
09 Nov 06:10    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


## BD-LRU only

In [7]:
gen_config(bd_lru_only=True, num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model R] will not be used in RecBole
09 Nov 06:16    INFO  ['run.py', '--model', 'R']
09 Nov 06:16    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


# No Conv1D

In [8]:
gen_config(disable_conv1d=True, num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model R] will not be used in RecBole
09 Nov 06:23    INFO  ['run.py', '--model', 'R']
09 Nov 06:23    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


# No FeedForward

In [9]:
gen_config(disable_ffn=True, num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model R] will not be used in RecBole
09 Nov 06:33    INFO  ['run.py', '--model', 'R']
09 Nov 06:33    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


In [10]:
import torch
import gc

gc.collect()

82

In [11]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache cleared.")

CUDA cache cleared.


# BERT4Rec

In [12]:
MODEL = 'B'
gen_config(num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model B] will not be used in RecBole
09 Nov 06:41    INFO  ['run.py', '--model', 'B']
09 Nov 06:41    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}


# SASRec

In [13]:
MODEL = 'S'
gen_config(num_epochs=EPOCHS, dataset=dataset)
!cd /root/UPDATED_STRUCTURE && python run.py --model $MODEL

Successfully wrote dynamically generated configuration to /root/UPDATED_STRUCTURE/config.yaml
command line args [--model S] will not be used in RecBole
09 Nov 06:42    INFO  ['run.py', '--model', 'S']
09 Nov 06:42    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/amazon-beauty
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
